- 참조 URL (DeepLab) : https://github.com/kazuto1011/deeplab-pytorch
- 참조 URL (COCO dataset) : https://github.com/nightrome/cocostuff

In [ ]:
### COCO dataset download ###
!wget --directory-prefix=downloads http://images.cocodataset.org/zips/val2017.zip
!mkdir -p dataset/images
!unzip downloads/val2017.zip -d dataset/images/

### annotation data(label) 인데, train data가 포함되어 용량이 커서 download 생략 ###
# !wget --directory-prefix=downloads http://calvin.inf.ed.ac.uk/wp-content/uploads/data/cocostuffdataset/stuffthingmaps_trainval2017.zip
# !mkdir -p dataset/annotations
# !unzip downloads/stuffthingmaps_trainval2017.zip -d dataset/annotations/

In [ ]:
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import torch

### numpy image를 torch tensor로 변환하는 함수 ###
def numpy2torch( image, use_cuda, normalize=True, diff=False ):
    if use_cuda:
        image_torch = torch.from_numpy(image).unsqueeze(0).transpose(1,3).transpose(2,3).cuda()
    else:
        image_torch = torch.from_numpy(image).unsqueeze(0).transpose(1,3).transpose(2,3)

    if normalize:    
        image_torch = image_torch.float() / 255.

    return image_torch

In [ ]:
model = torch.hub.load("kazuto1011/deeplab-pytorch", "deeplabv2_resnet101", pretrained='cocostuff164k', n_classes=182)
model.cuda()
model.eval()

In [ ]:
file_idx = 9

image_filenames = glob.glob( "dataset/images/val2017/*.jpg" ) # images 경로를 list로 저장
image_filenames = sorted( image_filenames, key = lambda x : int(x.split('/')[-1].split('.')[0]) ) # filename에 맞춰 sorting
image = cv2.imread( image_filenames[file_idx] )[..., ::-1] # BGR => RGB

plt.imshow( image )
plt.show()

# label_filenames = glob.glob( "dataset/annotations/val2017/*.png" ) # labels 경로를 list로 저장
# label_filenames = sorted( label_filenames, key = lambda x : int(x.split('/')[-1].split('.')[0]) ) # filename에 맞춰 sorting
# label = cv2.imread( label_filenames[file_idx] )[..., 0] # BGR => 1 channel

# plt.imshow( label )
# plt.show()

In [ ]:
### resize image & label ###
resize_image = cv2.resize( image, (513, 513) )
# resize_label = cv2.resize( label, (513, 513) )

### numpy array => torch tensor ###
torch_image = numpy2torch( resize_image, True, normalize=False )

In [ ]:
### inference ###
with torch.no_grad():
    pred = model( torch_image.float() )
    print(pred.shape)

    ### change shape (N, C, H, W) => (H, W, C) ###
    pred = pred[0].cpu().numpy().transpose(1,2,0)
    print(pred.shape)

In [ ]:
### probability => index ###
max_pred = np.argmax( pred, axis=-1 ).astype(np.uint8)

### resize prediction map ###
resize_pred = cv2.resize( max_pred, (513,513), interpolation=cv2.INTER_NEAREST )

In [ ]:
plt.figure( figsize=(8,8) )
plt.imshow( resize_image )
plt.imshow( resize_pred, alpha=0.5 )
plt.show()

plt.figure( figsize=(8,8) )
plt.imshow( resize_image )
plt.show()

plt.figure( figsize=(8,8) )
plt.imshow( resize_pred )
plt.show()

# plt.figure( figsize=(8,8) )
# plt.imshow( resize_label )
# plt.show()